## Optimize

There are 90 cards with all two-digit numbers on them:
$$10,11,12,…,98,99.$$

A player takes some of these cards simultaneously. For each card taken she gets $1. However, if the player takes two cards that add up to 100 (say, 23 and 77), she loses all the money. How much could she get? 

In mathematical language: What is the maximum number of two-digit integers (10,11,...,99) that one can select satisfying the following condition: no two different selected integers have sum 100?

In [2]:
pairs = [(i,j) for i in range(10,100) for j in range(i+1,100)]
invalidvalid_pairs = [pair for pair in pairs if sum(pair)==100]
totalinvalid = len(invalidvalid_pairs)

In [3]:
print(f'Maximum pairing: {90-totalinvalid}')

Maximum pairing: 50


Choose the maximal number of integers among 1..50 that can be selected if we are not allowed to select n and 2n at at same time.

In [4]:
map = [None for i in range(50)]
for i in range(50,0,-1):
    if map[i-1] is None:
        if i % 2 == 0:
            map[i//2 - 1] = 1
            map[i-1] = 0
        else:
            map[i-1] = 1
    
total = sum(map)
print(f'It can be selected a maximal amount of {total} numbers')


It can be selected a maximal amount of 33 numbers


Modify the code considered in the lectures to find out the number of solutions to the 8 queens puzzle. For example, as was explained in the videos, for the 4 queens puzzle there are 2 solutions.

In [5]:
import itertools as it

def is_solution(perm):
    for (i1, i2) in it.combinations(range(len(perm)), 2):
        if abs(i1 - i2) == abs(perm[i1] - perm[i2]):
            return False

    return True
count = 0
for perm in it.permutations(range(8)):
    if is_solution(perm):
        count +=1

print(f'The 8 queen problem has {count} solutions')

The 8 queen problem has 92 solutions


## Recursion

Imagine we have only 5- and 7-coins. One can prove that any large enough integer amount can be paid using only such coins. Yet clearly we cannot pay any of numbers 1, 2, 3, 4, 6, 8, 9 with our coins. What is the maximum amount that cannot be paid?



In [6]:
def coins(amount):
    if amount<0: return None
    if amount==5 or amount==7:
        return [amount]
   
    try7 = coins(amount-7)
    if try7 is not None:
        try7.append(7)
        return try7
    try5 = coins(amount-5)
    if try5 is not None:
        try5.append(5)
        return try5
    
    return None

for i in range(100):
    if coins(i) is None:
        max = i

print(f'Max value was {max}')

Max value was 23


# Project - Programming 15-Puzzle

This task (solving the 15-puzzle) contains  several parts. To find whether a solution exists, we need to know whether a given permutation is even.

In [10]:
def counting_transpositions(perm):
    n = len(perm)
    c=0
    for i in range(n):
        for j in range(i+1,n):
            if perm[i]>perm[j]:
                c+=1
    return c

def is_permutation(p):
  return (set(p)==set(range(len(p))))

def is_even_permutation(perm):
    return is_permutation(perm) and counting_transpositions(perm)%2==0


print (is_even_permutation([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))
print (is_even_permutation([0,2,1]))
print (is_even_permutation([1,2,3]))

True
False
False


Write a Python function solution(position) that gets a (solvable) position in 15-puzzle and outputs a sequence of moves that transforms it to a standard position.

<details>
  <summary><b>Hint used</b></summary>
  
  Write a search algorithm that just looks for this solution by trying all the paths. However, the search should be optimized: at each step of the search we try to extend some sequence of operations (as we did with backtracking for queens), and try all the one-step extensions in some order. To make search feasible, one should be careful about this order: the heuristic is to consider first the position that has a smaller "distance" to the desired position. Here the distance could be measured by the sum of distances between current and desirable positions of each piece. (The distance is measured by the number of moves needed to get from one position to the other; it is often called Manhattan distance.) My friend (Stanislav Shalunov) showed be a program of this type (using python tools for priority queues); it is much shorted (less that hundred lines instead of almost four hundred) than the program that uses the approach discussed earlier; it also gives solutions with much smaller number of moves (hundreds instead of thousands), but for some configuration it takes more time (due to the exponential nature of search algorithms in general)
  
</details>


In [8]:
import heapq

goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
map = {(i+1)%16 :i for i in range(16)}

def apply_map(position):
    return [map[i] for i in position]

moves = {
    "left": -1,
    "right": 1,
    "up": -4,
    "down": 4
}

#Checks if move is possible
def can_move(empty_pos, direction):
    if direction == "left" and empty_pos % 4 == 0:
        return False
    if direction == "right" and empty_pos % 4 == 3:
        return False
    if direction == "up" and empty_pos < 4:
        return False
    if direction == "down" and empty_pos > 11:
        return False
    return True

#Move piece to the desired position
def move_tile(position, empty_pos, direction):
    if can_move(empty_pos, direction):
        new_pos = empty_pos + moves[direction]
        new_position = list(position)
        new_position[empty_pos], new_position[new_pos] = new_position[new_pos], new_position[empty_pos]
        return tuple(new_position), new_pos
    return position, empty_pos

#Gets Manhattan distance
def manhattan_distance(position):
    distance = 0
    for i in range(16):
        if position[i] != 0:
            target_pos = position[i] - 1
            distance += abs(target_pos // 4 - i // 4) + abs(target_pos % 4 - i % 4)
    return distance

# Global variables to store paths and configurations
global_paths = []
explored_configurations = {}

#Solution by applying A* search
def solution(initial_position):
    global global_paths
    global explored_configurations

    assert is_even_permutation(apply_map(initial_position))==True

    if type(initial_position)==list: initial_position = tuple(initial_position)
    frontier = []
    heapq.heappush(frontier, (0 + manhattan_distance(initial_position), 0, initial_position, []))
    explored = set()

    while frontier:
        _, cost, position, path = heapq.heappop(frontier)

        if position in explored:
            continue

        explored.add(position)

        if position == goal_state:
            global_paths.append(path)
            explored_configurations[position] = path
            return path

        empty_pos = position.index(0)

        for direction in ["left", "right", "up", "down"]:
            new_position, new_empty_pos = move_tile(position, empty_pos, direction)

            if new_position != position and new_position not in explored:
                new_path = path + [new_position[empty_pos]]
                heapq.heappush(frontier, (cost + 1 + manhattan_distance(new_position), cost + 1, new_position, new_path))

        # Store explored configuration with its path
        if position not in explored_configurations:
            explored_configurations[position] = path

    return None


initial_position = [1,2,3, 4, 5, 6, 7, 8, 13, 9, 11,12, 10, 14, 15, 0]
solution(initial_position)


[15, 14, 10, 13, 9, 10, 14, 15]